In [1]:
import pandas as pd
from functions import assembleDf, epochG, epochsG
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2
import time

In [2]:
matches = pd.read_csv('../Data/matches_glicko2.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

In [3]:
matches

,tourney_date,winner_id,loser_id,tourney_id
0,1877-07-09,113987,114149,1877-540
1,1877-07-09,113987,113999,1877-540
2,1877-07-09,113987,113963,1877-540
3,1877-07-09,113963,114158,1877-540
4,1877-07-09,113999,113953,1877-540
...,...,...,...,...
876973,2022-12-26,207987,208519,2022-M-ITF-TUN-63A-2022
876974,2022-12-26,144932,209977,2022-M-ITF-TUN-63A-2022
876975,2022-12-26,209955,209942,2022-M-ITF-TUN-63A-2022
876976,2022-12-26,209079,210696,2022-M-ITF-TUN-63A-2022


In [4]:
# 32577 unique player ids in the matches DataFrame
len(set(list(matches['winner_id'].unique())+list(matches['loser_id'].unique())))

32577

Because memory constraints limit the batch size in epochsG, I will need to process incrementally.

In [5]:
# base batches.  can be further broken up by splitBatch
eg_batches_50 = [(s,f) for s,f in zip(range(0, 876_978-50_000,50_000),
                      range(50_000, 876978,50_000))] + [(850_000,876979)]
eg_batches_100 = [(s,f) for s,f in zip(range(0, 876_978-100_000,100_000),
                      range(100_000, 876978,100_000))] + [(800_000,876979)]

ratings_histories = []
players_dict = {}

In [6]:
def ceildiv(a, b):
    return -(a // -b)

In [7]:
def splitBatch(atuple, divisions):
    '''Break up batches into smaller pieces to allow epochsG to run.'''
    batch_size = ceildiv((atuple[1] - atuple[0]),divisions)
    if batch_size < (atuple[1] - atuple[0]):
        batches = [b for b in zip(range(atuple[0],atuple[1]-batch_size,batch_size),
             range(atuple[0]+batch_size,atuple[1],batch_size))]
        extra = [(batches[-1][1],atuple[1])]
        return batches + extra
    else:
        return atuple

eg_batches_50[0:3] runs...

In [8]:
%%time
for batch in eg_batches_50[0:3]:
    players_dict,rh = epochsG(matches[batch[0]:batch[1]],players_dict,365)
    ratings_histories += [rh]

CPU times: user 19 s, sys: 302 ms, total: 19.3 s
Wall time: 19.2 s


In [9]:
eg_batches_50[3]

(150000, 200000)

In [10]:
b = splitBatch(eg_batches_50[3],2); b0 = b[0]; b1 = b[1]
b

[(150000, 175000), (175000, 200000)]

In [11]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 2 s, sys: 34.5 ms, total: 2.03 s
Wall time: 2.03 s


In [12]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 2 s, sys: 16.3 ms, total: 2.01 s
Wall time: 2.01 s


In [13]:
splitBatch(eg_batches_50[4],3)

[(200000, 216667), (216667, 233334), (233334, 250000)]

In [14]:
b = splitBatch(eg_batches_50[4],4); 
b0 = b[0]; b1 = b[1]; b2 = b[2]; b3=b[3]
b

[(200000, 212500), (212500, 225000), (225000, 237500), (237500, 250000)]

In [15]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.18 s, sys: 15.5 ms, total: 1.2 s
Wall time: 1.21 s


In [16]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.07 s, sys: 6.86 ms, total: 1.07 s
Wall time: 1.08 s


In [17]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.08 s, sys: 10.8 ms, total: 1.09 s
Wall time: 1.08 s


In [18]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.14 s, sys: 10.2 ms, total: 1.15 s
Wall time: 1.14 s


In [19]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[5][0]:eg_batches_50[5][1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 3.85 s, sys: 28.2 ms, total: 3.88 s
Wall time: 3.89 s


%%time
players_dict,rh = epochsG(matches[eg_batches_50[6][0]:eg_batches_50[6][1]],players_dict,365)
ratings_histories += [rh]

In [20]:
b = splitBatch(eg_batches_50[6],8); 
b0 = b[0]; b1 = b[1]; b2 = b[2]; b3=b[3];
b4 = b[0]; b5 = b[1]; b6 = b[2]; b7=b[3]
b

[(300000, 306250),
 (306250, 312500),
 (312500, 318750),
 (318750, 325000),
 (325000, 331250),
 (331250, 337500),
 (337500, 343750),
 (343750, 350000)]

In [21]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 820 ms, sys: 9.12 ms, total: 829 ms
Wall time: 826 ms


In [22]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 881 ms, sys: 10.7 ms, total: 892 ms
Wall time: 888 ms


In [23]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 760 ms, sys: 12 ms, total: 772 ms
Wall time: 769 ms


In [24]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 843 ms, sys: 14 ms, total: 857 ms
Wall time: 848 ms


In [25]:
%%time
players_dict,rh = epochsG(matches[b4[0]:b4[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 827 ms, sys: 14.3 ms, total: 841 ms
Wall time: 832 ms


In [26]:
%%time
players_dict,rh = epochsG(matches[b5[0]:b5[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 901 ms, sys: 8.03 ms, total: 909 ms
Wall time: 908 ms


In [27]:
%%time
players_dict,rh = epochsG(matches[b6[0]:b6[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 747 ms, sys: 10.6 ms, total: 758 ms
Wall time: 755 ms


In [28]:
%%time
players_dict,rh = epochsG(matches[b7[0]:b7[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 832 ms, sys: 10.5 ms, total: 842 ms
Wall time: 839 ms


%%time

players_dict,rh = epochsG(matches[eg_batches_50[7][0]:eg_batches_50[7][1]],players_dict,365)
ratings_histories += [rh]

In [29]:
b = splitBatch(eg_batches_50[7],7); 
b0 = b[0]; b1 = b[1]; b2 = b[2]; b3 = b[3]; b4 = b[4]; b5 = b[5];
b6 = b[6]


In [30]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 890 ms, sys: 9.46 ms, total: 899 ms
Wall time: 898 ms


In [31]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 855 ms, sys: 10.7 ms, total: 866 ms
Wall time: 863 ms


In [32]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 940 ms, sys: 12.6 ms, total: 953 ms
Wall time: 947 ms


In [33]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 966 ms, sys: 12.7 ms, total: 979 ms
Wall time: 974 ms


In [34]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 936 ms, sys: 8.23 ms, total: 944 ms
Wall time: 941 ms


In [35]:
%%time
players_dict,rh = epochsG(matches[b4[0]:b4[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 856 ms, sys: 13.9 ms, total: 870 ms
Wall time: 862 ms


In [36]:
%%time
players_dict,rh = epochsG(matches[b5[0]:b5[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 969 ms, sys: 11.3 ms, total: 980 ms
Wall time: 974 ms


In [37]:
%%time
players_dict,rh = epochsG(matches[b6[0]:b6[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 968 ms, sys: 11.1 ms, total: 979 ms
Wall time: 977 ms


In [38]:
b = splitBatch(eg_batches_50[8],2); b0 = b[0]; b1 = b[1]
b

[(400000, 425000), (425000, 450000)]

In [39]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 2.09 s, sys: 15.3 ms, total: 2.11 s
Wall time: 2.11 s


In [40]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.91 s, sys: 15.4 ms, total: 1.92 s
Wall time: 1.92 s


In [41]:
b = splitBatch(eg_batches_50[9],8); b0 = b[0]; b1 = b[1]; 
b2 = b[2];b3 = b[3];b4 =b[4]; b5 = b[5]; b6 = b[6]; b7 = b[7]
b

[(450000, 456250),
 (456250, 462500),
 (462500, 468750),
 (468750, 475000),
 (475000, 481250),
 (481250, 487500),
 (487500, 493750),
 (493750, 500000)]

In [42]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 851 ms, sys: 9.96 ms, total: 861 ms
Wall time: 858 ms


In [43]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 958 ms, sys: 20.8 ms, total: 979 ms
Wall time: 967 ms


In [44]:
b = splitBatch(b2,2)
b2_0 = b[0]; b2_1 = b[1];

In [45]:
%%time
players_dict,rh = epochsG(matches[b2_0[0]:b2_0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 752 ms, sys: 21.8 ms, total: 774 ms
Wall time: 765 ms


In [46]:
%%time
players_dict,rh = epochsG(matches[b2_1[0]:b2_1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 773 ms, sys: 28.3 ms, total: 802 ms
Wall time: 787 ms


In [47]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 903 ms, sys: 14.9 ms, total: 918 ms
Wall time: 915 ms


In [48]:
%%time
players_dict,rh = epochsG(matches[b4[0]:b4[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 857 ms, sys: 15.8 ms, total: 872 ms
Wall time: 869 ms


In [49]:
%%time
players_dict,rh = epochsG(matches[b5[0]:b5[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 988 ms, sys: 15.9 ms, total: 1 s
Wall time: 995 ms


In [50]:
%%time
players_dict,rh = epochsG(matches[b6[0]:b6[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.01 s, sys: 10.4 ms, total: 1.02 s
Wall time: 1.02 s


In [51]:
%%time
players_dict,rh = epochsG(matches[b7[0]:b7[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 984 ms, sys: 10.9 ms, total: 995 ms
Wall time: 991 ms


In [52]:
b = splitBatch(eg_batches_50[10],2); b0 = b[0]; b1 = b[1]
b

[(500000, 525000), (525000, 550000)]

In [53]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.8 s, sys: 12.9 ms, total: 1.81 s
Wall time: 1.81 s


In [54]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.89 s, sys: 16.2 ms, total: 1.91 s
Wall time: 1.91 s


In [55]:
b = splitBatch(eg_batches_50[11],5); b0 = b[0]; b1 = b[1];
b2 = b[2]; b3 = b[3];b4 = b[4];
b

[(550000, 560000),
 (560000, 570000),
 (570000, 580000),
 (580000, 590000),
 (590000, 600000)]

In [56]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.13 s, sys: 14 ms, total: 1.15 s
Wall time: 1.14 s


In [57]:
b = splitBatch(b1,2)
b1_0 = b[0]; b1_1 = b[1];

In [58]:
%%time
players_dict,rh = epochsG(matches[b1_0[0]:b1_0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 961 ms, sys: 23.6 ms, total: 985 ms
Wall time: 968 ms


In [59]:
%%time
players_dict,rh = epochsG(matches[b1_1[0]:b1_1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 894 ms, sys: 27.1 ms, total: 922 ms
Wall time: 904 ms


In [60]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.09 s, sys: 22.2 ms, total: 1.11 s
Wall time: 1.11 s


In [61]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.12 s, sys: 16.7 ms, total: 1.14 s
Wall time: 1.14 s


In [62]:
%%time
players_dict,rh = epochsG(matches[b4[0]:b4[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.16 s, sys: 5.13 ms, total: 1.17 s
Wall time: 1.17 s


In [63]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[12][0]:eg_batches_50[12][1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 3.54 s, sys: 30.9 ms, total: 3.58 s
Wall time: 3.57 s


In [64]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[13][0]:eg_batches_50[13][1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 3.59 s, sys: 31.2 ms, total: 3.62 s
Wall time: 3.62 s


In [65]:
b = splitBatch(eg_batches_50[14],2); b0 = b[0]; b1 = b[1]
b

[(700000, 725000), (725000, 750000)]

In [66]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.8 s, sys: 22.6 ms, total: 1.82 s
Wall time: 1.84 s


In [67]:
b_1 = splitBatch(b1,2)
b_10=b_1[0];b_11=b_1[1]

In [68]:
%%time
players_dict,rh = epochsG(matches[b_10[0]:b_10[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.37 s, sys: 12.8 ms, total: 1.38 s
Wall time: 1.39 s


In [69]:
%%time
players_dict,rh = epochsG(matches[b_11[0]:b_11[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.29 s, sys: 16.2 ms, total: 1.31 s
Wall time: 1.3 s


In [70]:
b = splitBatch(eg_batches_50[14],2); b0 = b[0]; b1 = b[1]
b

[(700000, 725000), (725000, 750000)]

In [71]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.79 s, sys: 24.1 ms, total: 1.82 s
Wall time: 1.81 s


In [72]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.84 s, sys: 16.8 ms, total: 1.85 s
Wall time: 1.85 s


In [73]:
b = splitBatch(eg_batches_50[15],4); b0 = b[0]; b1 = b[1];
b2 = b[2]; b3 = b[3]


In [74]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.24 s, sys: 17 ms, total: 1.26 s
Wall time: 1.25 s


In [75]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.32 s, sys: 19.5 ms, total: 1.34 s
Wall time: 1.34 s


In [76]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.3 s, sys: 27.3 ms, total: 1.33 s
Wall time: 1.32 s


In [77]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.08 s, sys: 17.1 ms, total: 1.1 s
Wall time: 1.1 s


In [78]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[16][0]:eg_batches_50[16][1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 3.69 s, sys: 69.1 ms, total: 3.76 s
Wall time: 3.76 s


In [79]:
b = splitBatch(eg_batches_50[17],4)
b0 = b[0]; b1 = b[1]
b

[(850000, 856745), (856745, 863490), (863490, 870235), (870235, 876979)]

In [80]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 847 ms, sys: 18.3 ms, total: 865 ms
Wall time: 860 ms


In [81]:
matches[b0[0]:b0[1]]

,tourney_date,winner_id,loser_id,tourney_id
850000,2022-02-07,202400,202260,2022-M-ITF-GBR-04A-2022
850001,2022-02-07,202400,207784,2022-M-ITF-GBR-04A-2022
850002,2022-02-07,202260,208316,2022-M-ITF-GBR-04A-2022
850003,2022-02-07,105899,105181,2022-M-ITF-TUR-05A-2022
850004,2022-02-07,210472,208300,2022-M-ITF-GBR-04A-2022
...,...,...,...,...
856740,2022-05-09,133975,109699,2022-0460
856741,2022-05-09,200484,200095,2022-0460
856742,2022-05-09,106310,105430,2022-0460
856743,2022-05-09,105181,111576,2022-0460


In [82]:
b = splitBatch(eg_batches_50[17],4)
b0 = b[0]; b1 = b[1]; b2 = b[2]; b3 = b[3];
b

[(850000, 856745), (856745, 863490), (863490, 870235), (870235, 876979)]

In [83]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 867 ms, sys: 27.7 ms, total: 895 ms
Wall time: 880 ms


In [84]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 995 ms, sys: 17 ms, total: 1.01 s
Wall time: 1.01 s


In [85]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.04 s, sys: 18 ms, total: 1.06 s
Wall time: 1.06 s


In [86]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 983 ms, sys: 17.1 ms, total: 1 s
Wall time: 992 ms


In [87]:
ratingsrdHistory = {}
for rh in ratings_histories:
    ratingsrdHistory.update(rh)

In [88]:
rdHistory = {key: ratingsrdHistory[key][1] for key in ratingsrdHistory.keys()}
ratingsHistory = {key: ratingsrdHistory[key][0] for key in ratingsrdHistory.keys()}

In [89]:
ratingsHistory_df = assembleDf(ratingsHistory)
rdHistory_df = assembleDf(rdHistory)

In [90]:
padRow = pd.DataFrame({col: 1500 for col in ratingsHistory_df.columns}, 
                      index = [pd.Timestamp('1877-07-09T00')])
padRowrd = pd.DataFrame({col: 350 for col in ratingsHistory_df.columns}, 
                      index = [pd.Timestamp('1877-07-09T00')])

In [91]:
ratingsHistory_df= pd.concat([padRow,ratingsHistory_df],axis=0)
rdHistory_df = pd.concat([padRowrd,rdHistory_df],axis=0)

In [92]:
ratingsHistory_df = ratingsHistory_df.ffill(axis=0).fillna(1500)
rdHistory_df = rdHistory_df.ffill(axis=0).fillna(350)

In [93]:
ratingsHistory_df.to_csv('./ratings_histories_glicko2.csv')
rdHistory_df.to_csv('./rd_histories_glicko2.csv')

In [103]:
ratingsHistory_df.columns

DatetimeIndex(['1877-07-09', '1878-07-09', '1879-07-09', '1880-07-08',
               '1881-07-08', '1882-07-08', '1883-07-08', '1884-07-07',
               '1885-07-07', '1886-07-07',
               ...
               '2018-07-16', '2018-12-03', '2019-06-10', '2020-06-09',
               '2021-06-09', '2022-02-07', '2022-05-09', '2022-07-18',
               '2022-09-26', '2022-12-26'],
              dtype='datetime64[ns]', length=179, freq=None)

In [100]:
id_keys = [key[0] for key in  ratingsHistory.keys()]

In [101]:
131879 in id_keys

False